# Speech Processing - TD1

## Part 1: Qualitative analysis of audio files

**Question 1: In your opinion, why do we use this dataset?**

A1 - This dataset is one of the rare health and speech dataset available that presents those characteristics:
* All labels came from the diagnosis of clinicians;
* All parameters of speakers are equilibrate: age, gender (that reflect reality), control and depressed groups, education level;
* It has been recorded in real-life setting conditions in hospital, with laptop microphones;
* The language used in this dataset was Italian, a language pretty under-resourced;
* It can be easily re-used to perform classification task on speech features (like the authors did for depression in their article).

In [ ]:
# Task 3: Plot a spectrogram (to see quality of audio signal)
!pip install -U praat-parselmouth
import parselmouth
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

**Question 2: What are the characteristics of these audio files? (sampling rate, bitrate, duration, approximative Signal-to-Noise ratio?)**

In [ ]:
#Q2:
def signaltonoise(a, axis=0, ddof=0):
    """
    
    :param a: the speech waveform
    :param axis: the axis to compute the mean
    :param ddof: 
    :return: the signal-to-noise
    """
    a = np.asanyarray(a)
    m = a.mean(axis)
    sd = a.std(axis=axis, ddof=ddof)
    return np.where(sd == 0, 0, m/sd)


def question_2():
    """
    Answer to question: retrieve all requiered informations from hc, pt and all files then compute avg, std, min, max of
    these variables. 
    :return: None
    """
    # Task 3
    root_hc = r'C:\Users\Cam\PycharmProjects\td_speech_processing\Androids-Corpus\Interview-Task\audio\HC'
    root_pt = r'C:\Users\Cam\PycharmProjects\td_speech_processing\Androids-Corpus\Interview-Task\audio\PT'

    filenames_hc = [f'{root_hc}/{f}' for f in listdir(root_hc) if isfile(join(root_hc, f))]
    filenames_pt = [f'{root_pt}/{f}' for f in listdir(root_pt) if isfile(join(root_pt, f))]

    all_files = filenames_hc + filenames_pt

    durations = []
    snrs = []
    sample_rates = []
    bitrates = []

    print("All filenames_hc")
    for file in filenames_hc:
        # Load audio
        SPEECH_WAVEFORM, SAMPLE_RATE = torchaudio.load(file)
        sample_rates.append(SAMPLE_RATE)
        snr = signaltonoise(SPEECH_WAVEFORM.numpy()[0])
        snrs.append(snr)
        duration = SPEECH_WAVEFORM.shape[1] / SAMPLE_RATE
        metadata = torchaudio.info(file)
        bitrate = metadata.sample_rate * metadata.bits_per_sample / 1000
        bitrates.append(bitrate)
        durations.append(duration)

    print(f"Sample rate: {set(sample_rates)}\n"
          f"Bitrate: {set(bitrates)}\n"
          f"Mean duration: {round(np.mean(durations), 3)}'s\n"
          f"Std duration: {round(np.std(durations), 3)}'s\n"
          f"Min duration: {round(np.min(durations), 3)}'s\n"
          f"Max duration: {round(np.max(durations), 3)}'s\n"
          f"Mean snr: {round(np.mean(snrs), 7)}'s\n"
          f"Std snr: {round(np.std(snrs), 7)}'s\n"
          f"Min snr: {round(np.min(snrs), 7)}'s\n"
          f"Max snr: {round(np.max(snrs), 7)}'s\n"
          )


    durations = []
    snrs = []
    sample_rates = []
    bitrates = []
    print("All filenames_pt")
    for file in filenames_pt:
        # Load audio
        SPEECH_WAVEFORM, SAMPLE_RATE = torchaudio.load(file)
        sample_rates.append(SAMPLE_RATE)
        snr = signaltonoise(SPEECH_WAVEFORM.numpy()[0])
        snrs.append(snr)
        duration = SPEECH_WAVEFORM.shape[1] / SAMPLE_RATE
        metadata = torchaudio.info(file)
        bitrate = metadata.sample_rate * metadata.bits_per_sample / 1000
        bitrates.append(bitrate)
        durations.append(duration)

    print(f"Sample rate: {set(sample_rates)}\n"
          f"Bitrate: {set(bitrates)}\n"
          f"Mean duration: {round(np.mean(durations), 3)}'s\n"
          f"Std duration: {round(np.std(durations), 3)}'s\n"
          f"Min duration: {round(np.min(durations), 3)}'s\n"
          f"Max duration: {round(np.max(durations), 3)}'s\n"
          f"Mean snr: {round(np.mean(snrs), 7)}'s\n"
          f"Std snr: {round(np.std(snrs), 7)}'s\n"
          f"Min snr: {round(np.min(snrs), 7)}'s\n"
          f"Max snr: {round(np.max(snrs), 7)}'s\n")

    durations = []
    snrs = []
    sample_rates = []
    bitrates = []
    print("All files")
    for file in all_files:
        # Load audio
        SPEECH_WAVEFORM, SAMPLE_RATE = torchaudio.load(file)
        sample_rates.append(SAMPLE_RATE)
        snr = signaltonoise(SPEECH_WAVEFORM.numpy()[0])
        snrs.append(snr)
        duration = SPEECH_WAVEFORM.shape[1] / SAMPLE_RATE
        metadata = torchaudio.info(file)
        bitrate = metadata.sample_rate * metadata.bits_per_sample / 1000
        bitrates.append(bitrate)
        durations.append(duration)

    print(f"Sample rate: {set(sample_rates)}\n"
          f"Bitrate: {set(bitrates)}\n"
          f"Mean duration: {round(np.mean(durations), 3)}'s\n"
          f"Std duration: {round(np.std(durations), 3)}'s\n"
          f"Min duration: {round(np.min(durations), 3)}'s\n"
          f"Max duration: {round(np.max(durations), 3)}'s\n"
          f"Mean snr: {round(np.mean(snrs), 7)}'s\n"
          f"Std snr: {round(np.std(snrs), 7)}'s\n"
          f"Min snr: {round(np.min(snrs), 7)}'s\n"
          f"Max snr: {round(np.max(snrs), 7)}'s\n")